We would like to use SymPy methods to analyze course prerequisites. For this, every course will be defined as a variable (Symbol) and the prerequisite strings will be converted into SymPy expressions.

In [1]:
from sympy import *
import json
import re

def parse_course_name(full_name):
    return "".join(full_name.split(" - ")[0].split(" "))

with open("CoursePrerequisites.json", "r") as f:
    courses = json.loads(f.read())            

In [2]:
courses_dict = {} # course name -> course symbol
prereq_strings_dict = {} # course name -> prereq string

for (course_name, prerequisites) in courses.items():
    name = parse_course_name(course_name)
    courses_dict[name] = Symbol(name)
    prereq_strings_dict[name] = prerequisites

In [3]:
# prereqs_dict: course name -> prerequisite expression or none
prereqs_dict = {}

for (name, prerequisites_string) in prereq_strings_dict.items():
    if not prerequisites_string:
        continue
    else:
        try:
            prereqs_dict[name] = eval(prerequisites_string, courses_dict)
        except NameError:
            # there are some required classes that are not in the symbols dictionary yet (were not in the catalog)
            # TODO: these should be marked in some way
            
            # find all course names
            for new_course_name in re.findall("\w+", prerequisites_string):
                # create a symbol for the course if doesnt exist
                if new_course_name not in courses_dict:
                    courses_dict[new_course_name] = Symbol(new_course_name)
        prereqs_dict[name] = eval(prerequisites_string, courses_dict)


In [4]:
# check results
prereqs_dict["ACC301"]

ACC201 | MGMT202

In [5]:
# returns set of of (course >> prerequisites) for all needed prerequisites for a given 
# "set of prerequisite expressions of all courses a given course depends on in any way"
def get_all_prereqs(course, prerequisites, ret=set()):
    if not prerequisites:
        return ret
    ret.add(course >> prerequisites)
    
    # get sub-prerequisites for every course in the prerequisites
    for sub_course_symbol in prerequisites.atoms():
        if sub_course_symbol.name in prereqs_dict:
            get_all_prereqs(sub_course_symbol, prereqs_dict[sub_course_symbol.name], ret)
    return ret

In [6]:
# test recursion with a 2 level deep example
get_all_prereqs(courses_dict["ACC406"],prereqs_dict["ACC406"])

{Implies(ACC301, ACC201 | MGMT202), Implies(ACC406, ACC301)}

In [7]:
# checks if a given expression is satisfiable with given assumptions
def satisfiable_with_givens(expr, *givens):
    return satisfiable(And(expr, *givens));